In [142]:
import numpy as np
import pandas as pd
import glob
import os
from pathlib import Path
from itertools import product
import regex as re

DATA = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data'

#right, left, jump, fire
REMAP = {'Key.right': 'R', 
         'Key.left': 'L', 
         '\'d\'': 'J', 
         'Key.space': 'J',
         'Key.up': 'J'}

FPS = 60
N_FRAME = [2*FPS, 10*FPS, 60*FPS, 300*FPS]
BLOCK_TIME = ['2s', '10s', '1min', '5min']
MIN_OCCUR = [1, 2, 5, 20]


In [144]:
def count_overlapping(text, search_for):
    return len(re.findall(search_for, text, overlapped=True))

In [143]:
p2 = [''.join(item) for item in product('RrLlJj', repeat=2)]
p3 = [''.join(item) for item in product('RrLlJj', repeat=3)]
p4 = [''.join(item) for item in product('RrLlJj', repeat=4)]
p5 = [''.join(item) for item in product('RrLlJj', repeat=5)]
p6 = [''.join(item) for item in product('RrLlJj', repeat=6)]
p7 = [''.join(item) for item in product('RrLlJj', repeat=7)]
p8 = [''.join(item) for item in product('RrLlJj', repeat=8)]
p9 = [''.join(item) for item in product('RrLlJj', repeat=9)]
p10 = [''.join(item) for item in product('RrLlJj', repeat=10)]

list_patterns = [[], ['R', 'r', 'L', 'l', 'J', 'j'], p2, p3, p4, p5, p6, p7, p8, p9, p10]


In [114]:
df = pd.read_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/keylogs/ground_truth/2022-11-25-12_13_37.csv')
df

,FRAME,KEY,STATUS
0,20,Key.alt,DOWN
1,23,Key.tab,DOWN
2,30,Key.alt,UP
3,30,Key.tab,UP
4,186,Key.right,DOWN
...,...,...,...
6733,130332,Key.space,UP
6734,130668,Key.alt,DOWN
6735,130671,Key.tab,DOWN
6736,130677,Key.alt,UP


In [115]:
df_trim = df[(df['KEY'] == 'Key.left') | (df['KEY'] == 'Key.right') | (df['KEY'] == 'Key.space')]
df_trim

,FRAME,KEY,STATUS
4,186,Key.right,DOWN
6,302,Key.space,DOWN
7,311,Key.space,UP
8,330,Key.right,UP
10,353,Key.right,DOWN
...,...,...,...
6729,130284,Key.left,UP
6730,130295,Key.right,DOWN
6731,130323,Key.right,UP
6732,130323,Key.space,DOWN


In [116]:
#R -> Down, r -> Up
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'UP'), 'action'] = 'r'

df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'L'
df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'UP'), 'action'] = 'l'

df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'J'
df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'UP'), 'action'] = 'j'

df_trim

/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_34396/458184782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'


,FRAME,KEY,STATUS,action
4,186,Key.right,DOWN,R
6,302,Key.space,DOWN,J
7,311,Key.space,UP,j
8,330,Key.right,UP,r
10,353,Key.right,DOWN,R
...,...,...,...,...
6729,130284,Key.left,UP,l
6730,130295,Key.right,DOWN,R
6731,130323,Key.right,UP,r
6732,130323,Key.space,DOWN,J


In [140]:
#pattern size 2

freq2 = pd.DataFrame(p2, columns=['pattern'])
freq2['count'] = ''
freq2['frequency'] = ''

str = df_trim.action.str.cat()
patterns = freq2['pattern'].tolist()

for item in patterns:
    freq2.loc[freq2['pattern'] == item, 'count'] = str.count(item)

n_pattern = freq2['count'].sum()

for item in patterns:
    freq2.loc[freq2['pattern'] == item, 'frequency'] = freq2['count']/n_pattern * 100

freq2 = freq2.sort_values(by=['frequency'], ascending=False)
freq2

,pattern,count,frequency
29,Jj,1016,20.987399
34,jJ,560,11.567858
6,rR,326,6.734146
1,Rr,325,6.713489
31,jr,324,6.692832
4,RJ,322,6.651518
15,Ll,239,4.936996
8,rL,196,4.04875
18,lR,184,3.800868
30,jR,150,3.098533


In [137]:
#pattern size 3

freq3 = pd.DataFrame(p3, columns=['pattern'])
freq3['count'] = ''
freq3['frequency'] = ''

patterns = freq3['pattern'].tolist()

for item in patterns:
    freq3.loc[freq3['pattern'] == item, 'count'] = str.count(item)

n_pattern = freq3['count'].sum()

for item in patterns:
    freq3.loc[freq3['pattern'] == item, 'frequency'] = freq3['count']/n_pattern * 100

freq3 = freq3.sort_values(by=['frequency'], ascending=False)

In [146]:
#pattern size 3

freq3 = pd.DataFrame(p3, columns=['pattern'])
freq3['count'] = ''
freq3['frequency'] = ''

patterns = freq3['pattern'].tolist()

for item in patterns:
    freq3.loc[freq3['pattern'] == item, 'count'] = count_overlapping(str, item)

n_pattern = freq3['count'].sum()

for item in patterns:
    freq3.loc[freq3['pattern'] == item, 'frequency'] = freq3['count']/n_pattern * 100

freq3 = freq3.sort_values(by=['frequency'], ascending=False)

freq3

,pattern,count,frequency
178,JjJ,519,10.72314
209,jJj,518,10.702479
29,RJj,283,5.847107
175,Jjr,272,5.619835
6,RrR,180,3.719008
...,...,...,...
93,Lll,0,0.0
98,LJL,0,0.0
100,LJJ,0,0.0
102,LjR,0,0.0


In [147]:
dict_freq = dict()
str = df_trim.action.str.cat()


for i, list_pattern in enumerate(list_patterns[2:]):

    name = 'freq_{}'.format(i)

    freq = pd.DataFrame(list_pattern, columns=['pattern'])
    freq['count'] = ''
    freq['frequency'] = ''
    patterns = freq['pattern'].tolist()

    for item in patterns:
        freq.loc[freq['pattern'] == item, 'count'] = count_overlapping(str, item)

    n_pattern = freq['count'].sum()

    for item in patterns:
        freq.loc[freq['pattern'] == item, 'frequency'] = freq['count']/n_pattern * 100

    freq = freq.sort_values(by=['frequency'], ascending=False)

    dict_freq[name] = freq

dict_freq

In [145]:
candidates = dict()
best_timeframe_size = 0

for second_tick in range(2, 10, 0.1):
    n_frame = second_tick*FPS

    list_slices = list()
    slice_size = int(df_trim.FRAME.max()/n_frame)

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]
        list_slices.append(slice)


    for i, list_pattern in enumerate(list_patterns[2:5]):


        freq = pd.DataFrame(list_pattern, columns=['pattern'])
        freq['count'] = ''
        freq['frequency'] = ''
        patterns = freq['pattern'].tolist()

        for item in patterns:
            freq.loc[freq['pattern'] == item, 'count'] = count_overlapping(str, item)

        n_pattern = freq['count'].sum()

        for item in patterns:
            freq.loc[freq['pattern'] == item, 'frequency'] = freq['count']/n_pattern * 100

        freq = freq.sort_values(by=['frequency'], ascending=False)

        



3